# Transcoders for MLP Interpretability

This notebook uses **transcoders** to interpret a standard MLP classifier on MNIST.

### What we'll cover:
1. **MLP Training** - Train a standard 3-layer MLP (same architecture for comparison with bilinear MLP)
2. **Transcoder Training** - Train sparse autoencoders on MLP activations
3. **Feature Visualization** - Backproject transcoder features to input space
4. **Class-Specific Analysis** - Which features activate for each digit class
5. **Misclassification Analysis** - Understand why the model makes errors

### Key Idea
Transcoders learn a sparse, overcomplete representation of MLP activations:
```
MLP Layer Input → Sparse Encoding → MLP Layer Output (reconstructed)
```

The sparse features can be backprojected to pixel space for interpretation!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tqdm.auto import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

---
## Part 1: Data Loading

Load MNIST with optional Gaussian noise for regularization (matching the bilinear MLP notebook).

In [ ]:
class AddGaussianNoise:
    """Add Gaussian noise to images for regularization."""
    def __init__(self, mean=0., std=0.15):
        self.mean = mean
        self.std = std
    
    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.std + self.mean
        return tensor + noise


def get_dataloaders(batch_size=2048, noise_std=0.15, use_noise=True):
    """Create MNIST dataloaders with optional noise augmentation."""
    
    base_transforms = [transforms.ToTensor()]
    
    if use_noise and noise_std > 0:
        train_transforms = base_transforms + [AddGaussianNoise(std=noise_std)]
    else:
        train_transforms = base_transforms
    
    train_transform = transforms.Compose(train_transforms)
    test_transform = transforms.Compose(base_transforms)
    
    train_dataset = datasets.MNIST(
        root='./data', train=True, download=True, transform=train_transform
    )
    test_dataset = datasets.MNIST(
        root='./data', train=False, download=True, transform=test_transform
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader, train_dataset, test_dataset


# Load data
train_loader, test_loader, train_dataset, test_dataset = get_dataloaders(noise_std=0.15)
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Visualize samples with and without noise
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# Get a batch with noise (training)
sample_batch, labels = next(iter(train_loader))

for i in range(5):
    axes[0, i].imshow(sample_batch[i].squeeze(), cmap='gray')
    axes[0, i].set_title(f'Label: {labels[i].item()}')
    axes[0, i].axis('off')

# Load without noise for comparison
clean_loader, _, _, _ = get_dataloaders(use_noise=False)
clean_batch, clean_labels = next(iter(clean_loader))

for i in range(5):
    axes[1, i].imshow(clean_batch[i].squeeze(), cmap='gray')
    axes[1, i].set_title(f'Label: {clean_labels[i].item()}')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('With Noise', fontsize=12)
axes[1, 0].set_ylabel('Clean', fontsize=12)
plt.suptitle('MNIST Samples: Training with Noise vs Clean', fontsize=14)
plt.tight_layout()
plt.show()

---
## Part 2: Standard MLP Architecture

We use the **same architecture** as the bilinear MLP notebook for fair comparison:
- Input: 784 → Hidden: 512 → Output: 10

The key difference: Standard MLP uses **ReLU** activation, bilinear MLP uses **element-wise multiplication**.

In [ ]:
class MLP(nn.Module):
    """Standard 2-layer MLP: 784 -> 512 -> 10 (matching bilinear MLP architecture)
    
    Architecture comparison:
    - Standard MLP: Linear -> ReLU -> Linear
    - Bilinear MLP: Linear -> Bilinear(W*V) -> Linear
    """
    def __init__(self, input_dim=784, hidden_dim=512, num_classes=10):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        
        # Embedding layer (same as bilinear)
        self.embed = nn.Linear(input_dim, hidden_dim, bias=False)
        
        # ReLU activation (vs bilinear layer)
        self.activation = nn.ReLU()
        
        # Output head (same as bilinear)
        self.head = nn.Linear(hidden_dim, num_classes, bias=False)
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.xavier_uniform_(self.embed.weight)
        nn.init.xavier_uniform_(self.head.weight)
    
    def forward(self, x):
        if x.dim() > 2:
            x = x.view(x.size(0), -1)
        
        h = self.embed(x)       # Linear projection
        h = self.activation(h)  # ReLU nonlinearity
        logits = self.head(h)   # Classification head
        return logits
    
    def get_activations(self, x):
        """Get intermediate activations for transcoder training."""
        if x.dim() > 2:
            x = x.view(x.size(0), -1)
        
        pre_activation = self.embed(x)      # Before ReLU
        post_activation = self.activation(pre_activation)  # After ReLU
        
        return {
            'input': x,
            'pre_activation': pre_activation,
            'post_activation': post_activation
        }


model = MLP(hidden_dim=512).to(DEVICE)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\nArchitecture:")
print(f"  Embed: {model.embed.weight.shape}")
print(f"  Head: {model.head.weight.shape}")

---
## Part 3: Training the MLP

In [ ]:
def train_epoch(model, train_loader, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += data.size(0)
    
    return total_loss / total, 100. * correct / total


def evaluate(model, test_loader, device):
    """Evaluate model on test set."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target)
            
            total_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += data.size(0)
    
    return total_loss / total, 100. * correct / total


def train_model(model, train_loader, test_loader, epochs=20, lr=0.001, weight_decay=1.0):
    """Full training loop."""
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}
    
    pbar = tqdm(range(epochs), desc='Training')
    for epoch in pbar:
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, DEVICE)
        test_loss, test_acc = evaluate(model, test_loader, DEVICE)
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        pbar.set_postfix({
            'train_acc': f'{train_acc:.1f}%',
            'test_acc': f'{test_acc:.1f}%'
        })
    
    return history

In [ ]:
# Train the model with same hyperparameters as bilinear MLP
model = MLP(hidden_dim=512).to(DEVICE)

history = train_model(
    model, 
    train_loader, 
    test_loader,
    epochs=20,
    lr=0.001,
    weight_decay=1.0
)

print(f"\nFinal Test Accuracy: {history['test_acc'][-1]:.2f}%")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0].plot(history['test_loss'], label='Test', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].set_title('Training and Test Loss', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_acc'], label='Train', linewidth=2)
axes[1].plot(history['test_acc'], label='Test', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Accuracy (%)', fontsize=11)
axes[1].set_title('Training and Test Accuracy', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('Standard MLP Training on MNIST', fontsize=14)
plt.tight_layout()
plt.show()

---
## Part 4: Collect Activations for Transcoder Training

In [ ]:
def collect_activations(model, dataloader, device):
    """Collect MLP activations for transcoder training."""
    model.eval()
    
    all_inputs = []
    all_pre_acts = []
    all_post_acts = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Collecting activations"):
            images = images.to(device)
            activations = model.get_activations(images)
            
            all_inputs.append(activations['input'].cpu())
            all_pre_acts.append(activations['pre_activation'].cpu())
            all_post_acts.append(activations['post_activation'].cpu())
            all_labels.append(labels)
    
    return {
        'inputs': torch.cat(all_inputs),
        'pre_activation': torch.cat(all_pre_acts),
        'post_activation': torch.cat(all_post_acts),
        'labels': torch.cat(all_labels)
    }


# Collect activations from train and test sets
print("Collecting activations...")
train_acts = collect_activations(model, train_loader, DEVICE)
test_acts = collect_activations(model, test_loader, DEVICE)

print(f"\nTrain activations shape: {train_acts['post_activation'].shape}")
print(f"Test activations shape: {test_acts['post_activation'].shape}")

---
## Part 5: Transcoder Architecture

A **transcoder** is a sparse autoencoder that:
1. Takes MLP layer **inputs**
2. Encodes to a sparse, overcomplete representation
3. Reconstructs the MLP layer **outputs**

This differs from a standard SAE which reconstructs the same layer.

In [ ]:
class Transcoder(nn.Module):
    """Sparse transcoder: learns to predict layer outputs from layer inputs.
    
    Uses Top-K activation for sparsity (no auxiliary loss needed).
    
    Args:
        input_dim: Dimension of layer input
        output_dim: Dimension of layer output
        hidden_dim: Size of sparse hidden layer (overcomplete)
        k: Number of top activations to keep
    """
    def __init__(self, input_dim, output_dim, hidden_dim, k):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.k = k
        
        # Encoder: input -> sparse hidden
        self.encoder = nn.Linear(input_dim, hidden_dim)
        
        # Decoder: sparse hidden -> output
        self.decoder = nn.Linear(hidden_dim, output_dim)
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.kaiming_normal_(self.encoder.weight, nonlinearity='relu')
        nn.init.zeros_(self.encoder.bias)
        nn.init.normal_(self.decoder.weight, std=0.01)
        nn.init.zeros_(self.decoder.bias)
    
    def encode(self, x):
        """Encode input to sparse hidden representation."""
        z = self.encoder(x)
        
        # Top-K sparsity
        topk_vals, topk_idx = torch.topk(z, self.k, dim=-1)
        topk_vals = torch.relu(topk_vals)  # Only positive activations
        
        # Create sparse representation
        z_sparse = torch.zeros_like(z)
        z_sparse.scatter_(-1, topk_idx, topk_vals)
        
        return z_sparse
    
    def forward(self, x):
        """Forward pass: encode then decode."""
        z_sparse = self.encode(x)
        output = self.decoder(z_sparse)
        return output, z_sparse


def train_transcoder(transcoder, train_in, train_out, test_in, test_out, 
                     epochs=30, batch_size=256, lr=1e-3):
    """Train transcoder to predict layer outputs from inputs."""
    
    train_loader = DataLoader(
        TensorDataset(train_in, train_out), 
        batch_size=batch_size, 
        shuffle=True
    )
    
    optimizer = optim.Adam(transcoder.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.MSELoss()
    
    history = []
    
    for epoch in range(epochs):
        transcoder.train()
        train_loss = 0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            
            optimizer.zero_grad()
            pred, _ = transcoder(inputs)
            loss = criterion(pred, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(transcoder.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        scheduler.step()
        
        # Evaluate on test set
        transcoder.eval()
        with torch.no_grad():
            test_pred, _ = transcoder(test_in.to(DEVICE))
            test_loss = criterion(test_pred, test_out.to(DEVICE)).item()
        
        history.append({'train': train_loss / len(train_loader), 'test': test_loss})
        
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch+1}: Train MSE = {history[-1]['train']:.6f}, Test MSE = {test_loss:.6f}")
    
    return history

In [ ]:
# Train transcoder on the main layer (input -> post_activation)
# This matches the bilinear layer position

print("Training transcoder...")
print(f"  Input dim: {train_acts['inputs'].shape[1]}")
print(f"  Output dim: {train_acts['post_activation'].shape[1]}")
print(f"  Hidden dim: 2048 (4x expansion)")
print(f"  Sparsity k: 64")
print()

transcoder = Transcoder(
    input_dim=784,      # Input pixels
    output_dim=512,     # Post-activation (after ReLU)
    hidden_dim=2048,    # 4x overcomplete
    k=64                # Top-64 activations
).to(DEVICE)

tc_history = train_transcoder(
    transcoder,
    train_acts['inputs'],
    train_acts['post_activation'],
    test_acts['inputs'],
    test_acts['post_activation'],
    epochs=30
)

print("\nTranscoder training complete!")

In [ ]:
# Plot transcoder training loss
fig, ax = plt.subplots(figsize=(10, 4))

train_losses = [h['train'] for h in tc_history]
test_losses = [h['test'] for h in tc_history]

ax.plot(train_losses, label='Train', linewidth=2)
ax.plot(test_losses, label='Test', linewidth=2)
ax.set_xlabel('Epoch', fontsize=11)
ax.set_ylabel('MSE Loss', fontsize=11)
ax.set_title('Transcoder Training Loss', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Part 6: Visualize Transcoder Features

We can **backproject** transcoder features to input space to visualize what patterns each feature detects.

In [ ]:
def get_backprojected_features(transcoder, model):
    """Backproject transcoder encoder weights to input pixel space.
    
    The transcoder encoder learns: input -> sparse features
    So encoder.weight has shape (hidden_dim, input_dim) = (2048, 784)
    
    Each row is already in pixel space!
    """
    # Encoder weights are already in input space (784 = 28x28 pixels)
    encoder_weights = transcoder.encoder.weight.data.cpu()  # (2048, 784)
    
    return encoder_weights


def visualize_feature(weights, ax=None, title=None, cmap='RdBu_r'):
    """Visualize a feature as a 28x28 image."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(4, 4))
    
    img = weights.reshape(28, 28).numpy()
    img_smooth = gaussian_filter(img, sigma=0.5)
    
    vmax = np.abs(img_smooth).max()
    
    im = ax.imshow(img_smooth, cmap=cmap, vmin=-vmax, vmax=vmax)
    ax.axis('off')
    if title:
        ax.set_title(title, fontsize=10)
    
    return im


# Get backprojected features
backprojected = get_backprojected_features(transcoder, model)
print(f"Backprojected features shape: {backprojected.shape}")

In [ ]:
# Compute feature activations on test set
transcoder.eval()
with torch.no_grad():
    _, sparse_acts = transcoder(test_acts['inputs'].to(DEVICE))
    sparse_acts = sparse_acts.cpu()

# Find most active features (by mean activation)
feature_activations = (sparse_acts > 0).float().mean(dim=0)
top_features = torch.argsort(feature_activations, descending=True)

print(f"Feature activation statistics:")
print(f"  Most active feature fires {feature_activations[top_features[0]]:.1%} of the time")
print(f"  Median feature fires {feature_activations.median():.1%} of the time")
print(f"  Dead features (<1%): {(feature_activations < 0.01).sum().item()}")

In [ ]:
# Visualize top 20 most active features
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i, feat_idx in enumerate(top_features[:20]):
    feat_idx = feat_idx.item()
    activation_rate = feature_activations[feat_idx].item()
    
    visualize_feature(
        backprojected[feat_idx],
        ax=axes[i],
        title=f'Feature {feat_idx}\n(fires {activation_rate:.1%})'
    )

plt.suptitle('Top 20 Most Active Transcoder Features', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\nThese features show patterns the transcoder learned to detect in the input images.")
print("Blue = positive contribution, Red = negative contribution")

---
## Part 7: Class-Specific Feature Analysis

Which features are most active for each digit class?

In [ ]:
def get_class_feature_profiles(sparse_acts, labels):
    """Compute mean feature activations for each digit class."""
    n_features = sparse_acts.shape[1]
    class_profiles = torch.zeros(10, n_features)
    
    for digit in range(10):
        mask = labels == digit
        class_profiles[digit] = sparse_acts[mask].mean(dim=0)
    
    return class_profiles


# Compute class profiles
class_profiles = get_class_feature_profiles(sparse_acts, test_acts['labels'])
print(f"Class profiles shape: {class_profiles.shape}")

In [ ]:
# Visualize class activation heatmap
fig, ax = plt.subplots(figsize=(14, 6))

# Show first 100 most active features
top_100_features = top_features[:100]
heatmap_data = class_profiles[:, top_100_features].numpy()

im = ax.imshow(heatmap_data, aspect='auto', cmap='viridis')
ax.set_xlabel('Feature Index (sorted by activation frequency)', fontsize=11)
ax.set_ylabel('Digit Class', fontsize=11)
ax.set_yticks(range(10))
ax.set_title('Mean Feature Activation by Digit Class (Top 100 Features)', fontsize=12)
plt.colorbar(im, ax=ax, label='Mean Activation')

plt.tight_layout()
plt.show()

In [ ]:
# Find and visualize digit-specific features
def get_digit_specific_features(class_profiles, n_features=3):
    """Find features that are most specific to each digit.
    
    A digit-specific feature has high activation for one class
    and low activation for others.
    """
    digit_features = {}
    
    for digit in range(10):
        # Selectivity = activation for this digit - mean activation for other digits
        other_mean = class_profiles[torch.arange(10) != digit].mean(dim=0)
        selectivity = class_profiles[digit] - other_mean
        
        # Top selective features for this digit
        top_selective = torch.argsort(selectivity, descending=True)[:n_features]
        digit_features[digit] = top_selective.tolist()
    
    return digit_features


digit_specific = get_digit_specific_features(class_profiles, n_features=2)

# Visualize digit-specific features
fig, axes = plt.subplots(2, 10, figsize=(20, 5))

for digit in range(10):
    for i, feat_idx in enumerate(digit_specific[digit]):
        visualize_feature(
            backprojected[feat_idx],
            ax=axes[i, digit],
            title=f'F{feat_idx}' if i == 0 else f'F{feat_idx}'
        )
    axes[0, digit].set_title(f'Digit {digit}\nF{digit_specific[digit][0]}', fontsize=10)

axes[0, 0].set_ylabel('Top Feature', fontsize=10)
axes[1, 0].set_ylabel('2nd Feature', fontsize=10)

plt.suptitle('Most Digit-Specific Transcoder Features', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\nThese features fire most selectively for each digit class.")

---
## Part 8: Eigenvalue-like Analysis (Feature Importance)

We can analyze the **importance** of transcoder features, analogous to eigenvalue analysis in bilinear MLPs.

In [ ]:
# Compute feature importance via decoder weights
# Decoder weight magnitude indicates how much each feature contributes to output

decoder_weights = transcoder.decoder.weight.data.cpu()  # (512, 2048)
head_weights = model.head.weight.data.cpu()  # (10, 512)

# Feature importance for each class
# How much does feature f contribute to class c?
# contribution = sum over hidden dim of: head[c, h] * decoder[h, f]
feature_importance = head_weights @ decoder_weights  # (10, 2048)

print(f"Feature importance shape: {feature_importance.shape}")
print("(10 classes x 2048 features)")

In [ ]:
# Visualize feature importance spectrum for each digit (like eigenvalue spectra)
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for digit in range(10):
    importance = feature_importance[digit].numpy()
    
    # Sort by absolute importance
    sorted_idx = np.argsort(-np.abs(importance))
    sorted_importance = importance[sorted_idx[:50]]
    
    # Color by sign
    colors = ['blue' if v >= 0 else 'red' for v in sorted_importance]
    
    ax = axes[digit]
    ax.bar(range(50), sorted_importance, color=colors, alpha=0.7)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(f'Digit {digit}', fontsize=11)
    ax.set_xlabel('Feature Rank')
    if digit % 5 == 0:
        ax.set_ylabel('Importance')
    ax.set_xlim(-1, 50)

plt.suptitle('Feature Importance Spectra by Digit (Top 50)\n(Analogous to Eigenvalue Spectra in Bilinear MLPs)', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\nKey observation: Most features have near-zero importance!")
print("This LOW-RANK structure is similar to bilinear MLPs.")

In [ ]:
# Visualize most important positive and negative features for each digit
def visualize_digit_features(digit, feature_importance, backprojected, n_features=4):
    """Visualize top positive and negative features for a digit."""
    importance = feature_importance[digit]
    
    # Positive features (support classification as this digit)
    pos_features = torch.argsort(importance, descending=True)[:n_features]
    
    # Negative features (oppose classification as this digit)  
    neg_features = torch.argsort(importance, descending=False)[:n_features]
    
    fig = plt.figure(figsize=(14, 6))
    gs = fig.add_gridspec(2, n_features + 1, width_ratios=[1.5] + [1]*n_features)
    
    # Positive importance bar chart
    ax_pos = fig.add_subplot(gs[0, 0])
    pos_vals = importance[pos_features].numpy()
    ax_pos.barh(range(n_features), pos_vals, color='blue', alpha=0.7)
    ax_pos.set_xlabel('Importance')
    ax_pos.set_ylabel('Rank')
    ax_pos.set_title('Positive\n(supports digit)')
    ax_pos.invert_yaxis()
    
    # Positive feature visualizations
    for i, feat_idx in enumerate(pos_features):
        ax = fig.add_subplot(gs[0, i+1])
        visualize_feature(backprojected[feat_idx], ax=ax, 
                         title=f'F{feat_idx.item()}\nimp={importance[feat_idx]:.3f}')
    
    # Negative importance bar chart
    ax_neg = fig.add_subplot(gs[1, 0])
    neg_vals = importance[neg_features].numpy()
    ax_neg.barh(range(n_features), neg_vals, color='red', alpha=0.7)
    ax_neg.set_xlabel('Importance')
    ax_neg.set_ylabel('Rank')
    ax_neg.set_title('Negative\n(opposes digit)')
    ax_neg.invert_yaxis()
    
    # Negative feature visualizations
    for i, feat_idx in enumerate(neg_features):
        ax = fig.add_subplot(gs[1, i+1])
        visualize_feature(backprojected[feat_idx], ax=ax,
                         title=f'F{feat_idx.item()}\nimp={importance[feat_idx]:.3f}')
    
    plt.suptitle(f'Feature Analysis for Digit {digit}', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()


# Visualize for a few digits
for digit in [0, 3, 5, 8]:
    visualize_digit_features(digit, feature_importance, backprojected)

---
## Part 9: Misclassification Analysis

Use transcoders to understand why the model makes errors.

In [ ]:
# Find all misclassifications
model.eval()
with torch.no_grad():
    test_inputs = test_acts['inputs'].to(DEVICE)
    logits = model.head(model.activation(model.embed(test_inputs)))
    predictions = logits.argmax(dim=1).cpu()

true_labels = test_acts['labels']
misclassified_mask = predictions != true_labels
misclassified_idx = torch.where(misclassified_mask)[0]

print(f"Total misclassifications: {len(misclassified_idx)} / {len(true_labels)}")
print(f"Accuracy: {100 * (1 - len(misclassified_idx) / len(true_labels)):.2f}%")

In [ ]:
# Build confusion matrix
confusion = np.zeros((10, 10), dtype=int)
for idx in misclassified_idx:
    true_label = true_labels[idx].item()
    pred_label = predictions[idx].item()
    confusion[true_label, pred_label] += 1

# Plot confusion matrix
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(confusion, cmap='Reds')

for i in range(10):
    for j in range(10):
        if confusion[i, j] > 0:
            color = 'white' if confusion[i, j] > confusion.max()/2 else 'black'
            ax.text(j, i, confusion[i, j], ha='center', va='center', 
                   fontsize=10, color=color)

ax.set_xticks(range(10))
ax.set_yticks(range(10))
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('True', fontsize=12)
ax.set_title('Confusion Matrix (Errors Only)', fontsize=14)
plt.colorbar(im, label='Count')

plt.tight_layout()
plt.show()

# Print top confusion pairs
print("\nTop Confusion Pairs:")
print("=" * 40)
flat_idx = np.argsort(confusion.flatten())[::-1]
for idx in flat_idx[:10]:
    true_digit = idx // 10
    pred_digit = idx % 10
    count = confusion[true_digit, pred_digit]
    if count > 0:
        print(f"  {true_digit} → {pred_digit}: {count} errors")

In [ ]:
# Analyze a specific confusion pair
def analyze_confusion_pair(true_digit, pred_digit, test_acts, sparse_acts, 
                           predictions, true_labels, class_profiles, backprojected):
    """Analyze why the model confuses true_digit with pred_digit."""
    
    # Find errors for this pair
    pair_mask = (true_labels == true_digit) & (predictions == pred_digit)
    pair_idx = torch.where(pair_mask)[0]
    
    if len(pair_idx) == 0:
        print(f"No {true_digit} → {pred_digit} errors found")
        return
    
    print(f"\nAnalyzing {true_digit} → {pred_digit} confusion ({len(pair_idx)} errors)")
    print("=" * 60)
    
    # Get feature activations for these errors
    error_acts = sparse_acts[pair_idx]
    mean_error_acts = error_acts.mean(dim=0)
    
    # Compare to class profiles
    true_profile = class_profiles[true_digit]
    pred_profile = class_profiles[pred_digit]
    
    # Features that fired more than expected for predicted class
    excess_pred = mean_error_acts - true_profile
    misleading_features = torch.argsort(excess_pred, descending=True)[:5]
    
    # Features that should have fired for true class but didn't
    deficit_true = true_profile - mean_error_acts
    missing_features = torch.argsort(deficit_true, descending=True)[:5]
    
    print(f"\nMisleading features (fired unexpectedly): {misleading_features.tolist()[:3]}")
    print(f"Missing features (should have fired): {missing_features.tolist()[:3]}")
    
    # Visualize
    fig = plt.figure(figsize=(18, 10))
    
    # Row 1: Sample error images
    n_show = min(5, len(pair_idx))
    for i in range(n_show):
        ax = fig.add_subplot(3, 6, i + 1)
        img = test_acts['inputs'][pair_idx[i]].numpy().reshape(28, 28)
        ax.imshow(img, cmap='gray')
        ax.set_title(f'True: {true_digit}\nPred: {pred_digit}', fontsize=9)
        ax.axis('off')
    
    # Row 2: Misleading features
    ax = fig.add_subplot(3, 6, 7)
    ax.text(0.5, 0.5, 'Misleading\nFeatures', ha='center', va='center', fontsize=12)
    ax.axis('off')
    
    for i, feat_idx in enumerate(misleading_features[:5]):
        ax = fig.add_subplot(3, 6, 8 + i)
        visualize_feature(backprojected[feat_idx], ax=ax,
                         title=f'F{feat_idx.item()}')
    
    # Row 3: Missing features
    ax = fig.add_subplot(3, 6, 13)
    ax.text(0.5, 0.5, 'Missing\nFeatures', ha='center', va='center', fontsize=12)
    ax.axis('off')
    
    for i, feat_idx in enumerate(missing_features[:5]):
        ax = fig.add_subplot(3, 6, 14 + i)
        visualize_feature(backprojected[feat_idx], ax=ax,
                         title=f'F{feat_idx.item()}')
    
    plt.suptitle(f'Confusion Analysis: {true_digit} → {pred_digit}\n'
                 f'Row 1: Error samples | Row 2: Misleading features | Row 3: Missing features',
                 fontsize=12, y=1.02)
    plt.tight_layout()
    plt.show()


# Analyze top confusion pair
top_true = flat_idx[0] // 10
top_pred = flat_idx[0] % 10

analyze_confusion_pair(top_true, top_pred, test_acts, sparse_acts,
                      predictions, true_labels, class_profiles, backprojected)

In [ ]:
# Analyze top 3 confusion pairs
print("\n" + "="*70)
print("ANALYZING TOP 3 CONFUSION PAIRS")
print("="*70)

for idx in flat_idx[:3]:
    true_d = idx // 10
    pred_d = idx % 10
    if confusion[true_d, pred_d] > 0:
        analyze_confusion_pair(true_d, pred_d, test_acts, sparse_acts,
                              predictions, true_labels, class_profiles, backprojected)

---
## Part 10: Activation Statistics Dashboard

Comprehensive view of transcoder feature behavior.

In [ ]:
def plot_activation_dashboard(sparse_acts, labels):
    """Dashboard showing feature statistics."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Feature activation frequency histogram
    feature_freq = (sparse_acts > 0).float().mean(dim=0).numpy()
    axes[0, 0].hist(feature_freq, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].axvline(0.01, color='red', linestyle='--', linewidth=2, label='Dead threshold (1%)')
    axes[0, 0].set_xlabel('Activation Frequency', fontsize=11)
    axes[0, 0].set_ylabel('Number of Features', fontsize=11)
    axes[0, 0].set_title('Feature Activation Frequency Distribution', fontsize=12)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Dead vs Active features pie chart
    dead = (feature_freq < 0.01).sum()
    active = len(feature_freq) - dead
    colors = ['#2ecc71', '#e74c3c']
    axes[0, 1].pie([active, dead], 
                   labels=[f'Active\n({active})', f'Dead\n({dead})'],
                   autopct='%1.1f%%', 
                   colors=colors,
                   explode=(0.02, 0.02),
                   shadow=True,
                   textprops={'fontsize': 11})
    axes[0, 1].set_title('Feature Health', fontsize=12)
    
    # 3. Class selectivity distribution
    selectivity = []
    for feat_idx in range(sparse_acts.shape[1]):
        class_means = []
        for digit in range(10):
            mask = labels == digit
            class_means.append(sparse_acts[mask, feat_idx].mean().item())
        class_means = np.array(class_means)
        if class_means.sum() > 0:
            probs = class_means / (class_means.sum() + 1e-8)
            entropy = -np.sum(probs * np.log(probs + 1e-8))
            selectivity.append(np.log(10) - entropy)
        else:
            selectivity.append(0)
    
    axes[1, 0].hist(selectivity, bins=50, edgecolor='black', alpha=0.7, color='purple')
    axes[1, 0].set_xlabel('Class Selectivity Score', fontsize=11)
    axes[1, 0].set_ylabel('Number of Features', fontsize=11)
    axes[1, 0].set_title('Feature Class Selectivity (higher = more digit-specific)', fontsize=12)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Mean activation heatmap by class (first 100 features)
    class_mean_acts = np.zeros((10, sparse_acts.shape[1]))
    for digit in range(10):
        mask = labels == digit
        class_mean_acts[digit] = sparse_acts[mask].mean(dim=0).numpy()
    
    im = axes[1, 1].imshow(class_mean_acts[:, :100], aspect='auto', cmap='viridis')
    axes[1, 1].set_xlabel('Feature Index (first 100)', fontsize=11)
    axes[1, 1].set_ylabel('Digit Class', fontsize=11)
    axes[1, 1].set_yticks(range(10))
    axes[1, 1].set_title('Mean Activation by Class', fontsize=12)
    plt.colorbar(im, ax=axes[1, 1], label='Mean Activation')
    
    plt.suptitle('Transcoder Activation Statistics Dashboard', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return {'feature_freq': feature_freq, 'selectivity': np.array(selectivity), 'dead_count': dead}


# Plot dashboard
stats = plot_activation_dashboard(sparse_acts, test_acts['labels'])
print(f"\nStatistics:")
print(f"  Active features: {len(stats['feature_freq']) - stats['dead_count']}")
print(f"  Dead features: {stats['dead_count']}")

---
## Part 11: Feature Co-activation Analysis

In [ ]:
def plot_feature_coactivation(sparse_acts, top_k=50):
    """Heatmap showing which features tend to co-activate."""
    # Get most active features
    feature_freq = (sparse_acts > 0).float().mean(dim=0)
    top_features = torch.argsort(feature_freq, descending=True)[:top_k]
    
    # Compute correlation matrix
    selected_acts = sparse_acts[:, top_features].numpy()
    corr_matrix = np.corrcoef(selected_acts.T)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Correlation heatmap
    im1 = axes[0].imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0].set_title(f'Feature Co-activation Correlation\n(Top {top_k} Features)', fontsize=12)
    axes[0].set_xlabel('Feature Rank', fontsize=11)
    axes[0].set_ylabel('Feature Rank', fontsize=11)
    plt.colorbar(im1, ax=axes[0], label='Correlation')
    
    # Distribution of correlations
    upper_tri = corr_matrix[np.triu_indices(top_k, k=1)]
    axes[1].hist(upper_tri, bins=50, edgecolor='black', alpha=0.7, color='teal')
    axes[1].axvline(0, color='black', linestyle='-', linewidth=1)
    axes[1].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Strong positive (>0.5)')
    axes[1].axvline(-0.5, color='blue', linestyle='--', linewidth=2, label='Strong negative (<-0.5)')
    axes[1].set_xlabel('Correlation Coefficient', fontsize=11)
    axes[1].set_ylabel('Number of Feature Pairs', fontsize=11)
    axes[1].set_title('Distribution of Feature Correlations', fontsize=12)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle('Feature Co-activation Analysis', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print strongly correlated pairs
    strong_pairs = []
    for i in range(top_k):
        for j in range(i+1, top_k):
            if abs(corr_matrix[i, j]) > 0.5:
                strong_pairs.append((top_features[i].item(), top_features[j].item(), corr_matrix[i, j]))
    
    if strong_pairs:
        print(f"\nStrongly correlated feature pairs (|r| > 0.5):")
        for f1, f2, r in sorted(strong_pairs, key=lambda x: -abs(x[2]))[:10]:
            print(f"  F{f1} <-> F{f2}: r = {r:.3f}")


plot_feature_coactivation(sparse_acts, top_k=50)

---
## Part 12: t-SNE Visualization of Sparse Features

In [ ]:
from sklearn.manifold import TSNE

def plot_sparse_tsne(sparse_acts, labels, n_samples=2000):
    """t-SNE visualization of sparse activations colored by digit class."""
    # Subsample for speed
    n_total = len(sparse_acts)
    indices = np.random.choice(n_total, min(n_samples, n_total), replace=False)
    X = sparse_acts[indices].numpy()
    y = labels[indices].numpy()
    
    # Fit t-SNE
    print(f"Computing t-SNE for {len(indices)} samples...")
    tsne = TSNE(n_components=2, perplexity=30, random_state=SEED, max_iter=1000)
    X_embedded = tsne.fit_transform(X)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    scatter = ax.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y, cmap='tab10', alpha=0.6, s=15)
    
    # Colorbar with digit labels
    cbar = plt.colorbar(scatter, ax=ax, ticks=range(10))
    cbar.set_label('Digit Class', fontsize=11)
    
    # Add class centroids
    for digit in range(10):
        mask = y == digit
        if mask.sum() > 0:
            centroid = X_embedded[mask].mean(axis=0)
            ax.annotate(str(digit), centroid, fontsize=14, fontweight='bold',
                       ha='center', va='center',
                       bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black', alpha=0.8))
    
    ax.set_title('t-SNE of Transcoder Sparse Activations', fontsize=14, fontweight='bold')
    ax.set_xlabel('t-SNE Dimension 1', fontsize=11)
    ax.set_ylabel('t-SNE Dimension 2', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return X_embedded, y


# Plot t-SNE
print("="*70)
print("t-SNE VISUALIZATION OF SPARSE FEATURES")
print("="*70)
print("(Shows how digits cluster in transcoder feature space)\n")

tsne_embedding, tsne_labels = plot_sparse_tsne(sparse_acts, test_acts['labels'])

---
## Part 13: Low-Rank Structure Analysis

Similar to bilinear MLPs, we can check if few features capture most of the model's behavior.

In [ ]:
def evaluate_feature_ablation(model, transcoder, test_loader, n_features_list, device):
    """Evaluate accuracy when using only top-n features per sample.
    
    This tests the low-rank structure: can few features explain model behavior?
    """
    model.eval()
    transcoder.eval()
    
    results = {}
    
    for n_features in tqdm(n_features_list, desc="Testing feature ablation"):
        correct = 0
        total = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                data = data.view(data.size(0), -1).to(device)
                target = target.to(device)
                
                # Get sparse activations with modified k
                z = transcoder.encoder(data)
                topk_vals, topk_idx = torch.topk(z, n_features, dim=-1)
                topk_vals = torch.relu(topk_vals)
                z_sparse = torch.zeros_like(z)
                z_sparse.scatter_(-1, topk_idx, topk_vals)
                
                # Decode
                reconstructed = transcoder.decoder(z_sparse)
                
                # Classify using reconstructed activations
                logits = model.head(reconstructed)
                pred = logits.argmax(dim=1)
                correct += pred.eq(target).sum().item()
                total += target.size(0)
        
        results[n_features] = 100 * correct / total
    
    return results


# Test with different numbers of features
n_features_list = [1, 2, 4, 8, 16, 32, 64, 128, 256]
ablation_results = evaluate_feature_ablation(model, transcoder, test_loader, n_features_list, DEVICE)

print("\nFeature Ablation Results:")
print("=" * 40)
for n, acc in ablation_results.items():
    print(f"  {n:3d} features: {acc:.2f}%")

In [ ]:
# Plot ablation results
fig, ax = plt.subplots(figsize=(10, 6))

n_list = list(ablation_results.keys())
acc_list = list(ablation_results.values())

ax.plot(n_list, acc_list, 'bo-', linewidth=2, markersize=8)
ax.axhline(y=history['test_acc'][-1], color='r', linestyle='--', 
           label=f'Full model ({history["test_acc"][-1]:.1f}%)')

ax.set_xlabel('Number of Active Features (Top-K)', fontsize=12)
ax.set_ylabel('Test Accuracy (%)', fontsize=12)
ax.set_title('Accuracy vs. Number of Active Transcoder Features\n(Low-Rank Structure Analysis)', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log', base=2)

# Add annotations
for n, acc in list(ablation_results.items())[:5]:
    ax.annotate(f'{acc:.1f}%', (n, acc), textcoords='offset points', 
                xytext=(5, 5), fontsize=9)

plt.tight_layout()
plt.show()

print("\nKey insight: The transcoder exhibits LOW-RANK structure!")
print("A small number of features captures most of the model's behavior.")

---
## Part 14: Summary

### What We've Demonstrated:

1. **Standard MLPs can be interpreted using transcoders** - Sparse autoencoders reveal interpretable features

2. **Features are class-specific** - Different features fire for different digits

3. **Low-rank structure exists** - Few features capture most model behavior (similar to bilinear MLPs!)

4. **Misclassifications can be analyzed** - We can see which features caused errors

### Comparison with Bilinear MLPs:

| Aspect | Bilinear MLP | Standard MLP + Transcoder |
|--------|--------------|---------------------------|
| Interpretability Method | Eigendecomposition | Sparse Autoencoder |
| Features | Eigenvectors | Encoder weights |
| Importance | Eigenvalues | Decoder weights / activation frequency |
| Low-rank structure | ✓ Yes | ✓ Yes |
| Requires extra training | No | Yes (transcoder) |
| Works with standard activations | No | Yes |

In [ ]:
# Final summary
print("="*70)
print("EXPERIMENT SUMMARY")
print("="*70)

print(f"\nModel Architecture:")
print(f"  Input: 784 -> Hidden: 512 -> Output: 10")
print(f"  Activation: ReLU")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

print(f"\nTranscoder Architecture:")
print(f"  Input: 784 -> Hidden: 2048 -> Output: 512")
print(f"  Sparsity: Top-64")
print(f"  Parameters: {sum(p.numel() for p in transcoder.parameters()):,}")

print(f"\nPerformance:")
print(f"  MLP Test Accuracy: {history['test_acc'][-1]:.2f}%")
print(f"  Transcoder Final MSE: {tc_history[-1]['test']:.6f}")

print(f"\nLow-Rank Structure:")
print(f"  With 16 features: {ablation_results[16]:.2f}%")
print(f"  With 32 features: {ablation_results[32]:.2f}%")
print(f"  With 64 features: {ablation_results[64]:.2f}%")

print(f"\nKey Insight:")
print(f"  Transcoders enable interpretability of standard MLPs")
print(f"  Features are sparse, class-specific, and low-rank")
print(f"  Similar structure to bilinear MLPs emerges!")
print("="*70)

In [ ]:
# Save model and transcoder
torch.save({
    'model': model.state_dict(),
    'transcoder': transcoder.state_dict(),
    'mlp_accuracy': history['test_acc'][-1],
    'training_history': history,
    'transcoder_history': tc_history,
    'ablation_results': ablation_results,
}, 'transcoder_interpretability_model.pth')

print("Model and transcoder saved to 'transcoder_interpretability_model.pth'")